# Matrix factorisation hyperparameter selection


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.model_selection import KFold
%matplotlib inline
import factor

In [ ]:
n_models = 20
n_tasks = 30
true_dims = 5
noise = 0.1
H_true = np.random.rand(n_models, true_dims)
W_true = np.random.rand(true_dims, n_tasks)
X = H_true @ W_true / 2.5 + noise * np.random.rand(n_models, n_tasks)
plt.imshow(X)
plt.colorbar()

In [ ]:
# We can do crossval on this later
mask = np.random.rand(*X.shape) > 0.8
Xm = X.copy()
Xm[mask] = 0
plt.imshow(Xm)
plt.title("X with missing data");

In [ ]:
# How good was the reconstruction?
W, H, mu = factor.nmf_init(Xm, mask, true_dims)
#W, H, mu = factor.NMF(Xm, true_dims, mask)
F = W@H + mu
plt.plot(F[~mask], X[~mask], 'bo', alpha=0.3, label="Observed data")
plt.plot(F[mask], X[mask], 'ro', alpha=0.9, label="Held-out data")
plt.title("Reconstructed vs truth")
plt.legend()
plt.show()

In [ ]:
# Use crossvalidation to select the number of latent factors...
n_factors = np.arange(1, 9)

n_folds = 10
repeats = 10
fit_err = []  # seeing all the data
heldout_err = []  # crossval
heldout_std = []
method = factor.PCA
#method = factor.NMF

n_models, n_tasks = X.shape
indices = np.arange(n_models * n_tasks)


for dims in n_factors:
    print(f"Evaluating {dims} dimensions...")
    MSEs = []  # Samples losses

    for r in range(repeats):   
        kf = KFold(n_splits=n_folds, shuffle=True, random_state=42+r)
        for train_idx, test_idx in kf.split(indices):
            mask = np.zeros((n_models, n_tasks), bool)
            mask.flat[test_idx] = True
            assert all(mask.shape[1] != mask.sum(axis=1))       
            W, H, mu = method(X, dims, mask)
            R = W@H + mu  # reconstruction
            MSEs.append(np.mean((R[mask] - X[mask])**2))
    
    heldout_err.append(np.mean(MSEs))  # should be the same
    heldout_std.append(np.std(MSEs) / np.sqrt(len(MSEs)))
                       
    # Control - no masking
    mask = np.zeros((n_models, n_tasks), bool)
    W, H, mu = method(X, dims)
    recon = mu + W@H
    fit_err.append(np.mean((recon - X)**2))

heldout_err = np.array(heldout_err)
heldout_std = np.array(heldout_std)

method_name = method.__name__
plt.plot(n_factors, fit_err, "bo-", label="Fit Error")
plt.plot(n_factors, heldout_err, "ro-", label="Heldout Error")
plt.fill_between(n_factors, heldout_err-2*heldout_std, heldout_err+2*heldout_std, color="r", alpha=0.25)
plt.legend()
plt.title(method_name + " factor selection")
plt.show()

## TODO: Tune Regularisation hyperparameters

## TODO: Regularised PCA